# Analysis of refactoring and grouping variables

In [ ]:
import os
from pathlib import Path
from typing import Tuple

from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import pandas as pd
import re, yaml
from scipy.io.arff import loadarff
from scipy.io.arff._arffread import MetaData

In [ ]:
DATA_PATH = Path("../data/file65ef3a759daf.arff")
CONFIG_HARDCODED = Path("..") / "config" / "dict_hard.yaml"
CONFIG_MAKE = Path("..") / "config" / "dict.yaml"

## Data Preprocessing Functions

In [ ]:
def load_data(path_: Path) -> Tuple[pd.DataFrame, MetaData]:
    """Loads the .arff file (incl. metadata) and converts to utf-8.

    Parameters
    -------
    path_ : Path
            Path of the data.

    Returns
    -------
    data : pd.DataFrame
            Data as a dataframe.
    meta : scipy.io.arff._arffread.Metadata
            Metadata of the dataset.
    """
    # load df and metadata from .arff
    data, meta = loadarff(path_)
    data = pd.DataFrame(data)

    # remove b string from data
    str_df = data.select_dtypes([object])
    str_df = str_df.reset_index().melt(id_vars="index").set_index("index")
    str_df["value"] = str_df["value"].str.decode("utf-8")

    # rename the 'value' column to avoid conflicts and perform pivot
    str_df = str_df.rename(columns={"value": "decoded_value"})
    str_df = pd.pivot_table(
        str_df, columns="variable", values="decoded_value", index="index", aggfunc=lambda x: x
    )

    # reset both the column and index names to None
    str_df = str_df.rename_axis(index=None, columns=None)

    # merge str and non-str columns
    data = pd.concat([str_df, data.select_dtypes(exclude=[object])], axis=1)

    return data, meta


def create_yaml(dictionary: dict, yaml_path: Path, column: str) -> None:
    """Creates yaml file from dictionary.

    Parameters
    -------
    dictionary : dict
            Dictionary that is nested and appended.
    yaml_path : Path
        Path of the output yaml file.
    column : str
            Column name that is referenced for the mapping.

    Returns
    -------
    None
    """
    # created nested dict with column as key
    dictionary = {column: dictionary}

    # check if file already exists and update the dict inside by appending
    if os.path.isfile(yaml_path):
        with open(yaml_path, "r") as yamlfile:
            cur_yaml = yaml.load(yamlfile, Loader=yaml.FullLoader)
            cur_yaml.update(dictionary)
            dictionary = cur_yaml

    # save dict into (new) file
    with open(yaml_path, "w") as yamlfile:
        yaml.safe_dump(dictionary, yamlfile)


def read_yaml(path: Path) -> dict:
    """Reads yaml file from given path and returns as dict.

    Parameters
    -------
    path : Path
            Path of the respective yaml file.

    Returns
    -------
    make_match_dictionary : dict
            Yaml file loaded as dict.
    """
    with open(path, "r") as yaml_file:
        dictionary = yaml.load(yaml_file, Loader=yaml.FullLoader)

    return dictionary

In [ ]:
def clean_string(s: str) -> str:
    """Cleans the string by converting to lowercase and removing alphabetical values.

    Parameters
    -------
    s : str
        String that should be changd.

    Returns
    -------
    s : str
        Converted string.
    """
    # Remove non-alphanumeric characters and convert to lowercase
    s = re.sub(r"[^a-zA-Z0-9\s]", "", s)
    s = s.lower()
    return s


def get_best_match(value: str, choices: dict, threshold: int = 50) -> str:
    """Finds the best match for a given value within the choices.

    Parameters
    -------
    value : str
        The value to find the best match for.
    choices : dict
        A dicitionary in which the values
    choices : dict
        A dicitionary in which the values of a row are similar words or
        choices for matching with the respective value of the column.
    threshold : int
        The threshold set for the similarity score between the
        column value and the choices. Defaults to 50.

    Returns
    -------
    best_match : str
        The best match for the input value.
    """
    if not value or len(value) < 3:  # Skip empty strings and very short strings
        return value

    # Clean the input value
    value = clean_string(value)
    # Clean the choices
    cleaned_choices = [clean_string(choice) for choice in choices]

    # Use fuzz.token_set_ratio for better token matching
    best_match, score = process.extractOne(value, cleaned_choices, scorer=fuzz.token_set_ratio)

    if score >= threshold:
        best_match = best_match
    else:
        best_match = value

    return best_match


# Function to find the best match
def replace_with_best_match(
    df: pd.DataFrame, column_name: str, choices: dict, threshold: int = 50
) -> pd.DataFrame:
    """Finds best match between value of the dataframe & of dictionary.

    Parameters
    -------
    df : pd.DataFrame
        The input DataFrame where one column should.
    column_name : str
        The name of column that should be replaced with it's best match.
    choices : dict
        A dicitionary in which the values of a row are similar words or
        choices for matching with the respective value of the column.
    threshold : int
        The threshold set for the similarity score between the
        column value and the choices. Defaults to 50.

    Returns
    -------
    data : pd.DataFrame
        Transofrmed dataframe.
    """
    df[column_name] = df[column_name].apply(lambda x: get_best_match(x, choices, threshold))

    return df

In [ ]:
def replace_with_hard(data: pd.DataFrame, dict_hard: dict, column: str) -> pd.DataFrame:
    """Replaces existing categories based on a hard encoded dictionary.

    Parameters
    -------
    data : pd.DataFrame
        Dataframe to transform.
    dict_hard : dict
        Dict that will be used to change the values of a column.
    column : str
        The column that will be changed.

    Returns
    -------
    data : pd.DataFrame
        Transofrmed dataframe.
    """
    inv_map = {val: k for k, v in dict_hard.items() for val in v}
    data[column] = data[column].replace(inv_map)
    return data

In [ ]:
def categorize_top_n(df: pd.DataFrame, column_name: str, n: int) -> pd.DataFrame:
    """Keep top n classes & missing values and set rest of categories as other.

    Args:
        df (pd.DataFrame): The df used
        column_name (str): The name of the column that should be recategorized
        n (_type_): _description_

    Returns:
        pd.DataFrame: returns dataframe with limited number of classes
    """
    # get the value counts for the specified column
    value_counts = df[column_name].value_counts()

    # get top n categories
    top_n_categories = value_counts.index[:n].tolist()
    top_n_categories.append("?")

    # repalce categories not in top n with 'Other'
    df[column_name] = df[column_name].apply(lambda x: x if x in top_n_categories else "other")

    return df

## Preprocess Data

In [ ]:
data, meta = load_data(DATA_PATH)

In [ ]:
meta

In [ ]:
data["Make"].value_counts()

In [ ]:
data["Color"].value_counts()

In [ ]:
data["VehicleType"].value_counts()

#### Merging categories for feature "Make"

We observe that in the feature "Make" many of the categories in this case car brands/ car manufacturers are misspelled or varying words are used for the same brand. Therefore we need to marge these similar words so that they have the same category.  


In [ ]:
car_variations = {
    "Toyota": ["TOYOTA", "TOYOTA", "TOYTA", "TOYT", "TOYOYA", "TOTY", "TOYO"],
    "Honda": ["HONDA", "HONNDA", "HNDA", "HON", "HOND", "HONDA 4D"],
    "Hyundai": [
        "HYUNDAI",
        "HUNDAI",
        "HYANDAI",
        "HUNDAII",
        "HYUND",
        "HYUN",
        "HYUNDIA",
        "HYUNVA",
        "HYUANDAI",
        "HYNDIA",
        "HYINDAI",
        "HYUANDAI",
    ],
    "Nissan": [
        "NISSAN",
        "NISSON",
        "NISAN",
        "NISS",
        "NISSAB",
        "NIIS",
        "NISSAM",
        "NISSN",
        "NISSS",
        "NSSIAN",
        "NISSVAL1996",
        "NISSVAL1999",
    ],
    "Ford": ["FORD"],
    "Freightliner": [
        "FRGHT",
        "FRHT",
        "FRIGHTLINER",
        "FREI",
        "FREIGHT",
        "FREIGHTTK",
        "FREIGHTLINRT",
        "FREIGHTLINER",
        "FREIGHTTK",
    ],
    "Mazda": [
        "MAZDA",
        "MAZAD",
        "MAZA",
        "MADZA",
        "MAD",
        "MADZ",
        "MADZDA",
        "MAZVA",
        "MAZVA",
        "MAZS",
        "MAZDVAL2010",
        "MAZDVA",
        "MADZA",
        "MAZDVAL2010",
        "MAZSA",
        "MAZDA",
        "MAZDVA",
        "MAZVAL2010",
        "MAZDA",
        "MAZSA",
        "MAZDA",
        "MAZDA",
        "MAZVAL2010",
        "MAZVA",
        "MAZVAL2010",
        "MAZDA",
        "MAZDVAL2010",
    ],
    "Chrysler": [
        "CHRYSLER",
        "CHRY",
        "CHRSYLER",
        "CHRYS",
        "CHYRSLER",
        "CHRUSLER",
        "CHY",
        "CHYR",
        "CHRYSLTER",
    ],
    "Volkswagen": [
        "VOLKSWAGEN",
        "VOLKS",
        "VW",
        "VOLK",
        "VOLKSWAGON",
        "VOLKVAL2012",
        "VOLVAL2012",
        "VOLKS",
        "VOLKE",
        "VOLKW",
        "VOLKSW",
        "VOLKSWAG",
        "VOLKSWA",
        "VOLKSW",
        "VOLKSWA",
        "VOLKSWAGGON",
        "VOLKSWAGONQ",
        "VOLKSWAGGON",
        "VWAGON",
        "VOLK",
        "VOLVSWAGEN",
        "VOLKSWAGOM",
        "VOLKA",
        "VWOLKS",
        "VOLKSWAGO",
        "VOKS",
        "VOLKS",
        "VWAGON",
    ],
    "Kia": ["KIA", "KARA", "GENEVAL2003", "KYMCO"],
    "Volvo": ["VOLV0", "VOLKS", "VOLVO", "VOLVO TK", "VOLVOT", "VOLVO SW"],
    "Chevrolet": [
        "CHEVROLET",
        "CHEVEROLET",
        "CHEV",
        "CHEVY",
        "CHEVEROLET",
        "CEHVY",
        "CHECROLET",
        "CHE V",
        "CHEVOLRET",
        "CHEVRLET",
        "CHEVROLETE",
        "CHEVR",
        "CHEVROLT",
        "CHEVROLER",
        "CHEVROLERT",
        "CHECEVY",
        "CHECHY",
        "CHECY",
        "CEVEROLET",
        "CEVY",
        "CHEVORLET",
        "CHEVTOLET",
        "CHEVY GEO",
        "CHECROLET",
    ],
    "Infiniti": ["INFINITI", "INFI", "INFINIT", "INFIITI", "INFINIITI"],
    "Lexus": ["LEXUS", "LEXS", "LEXUS4D", "LEXU", "LEXSUS", "LEVUS", "LRXUS"],
    "Acura": ["ACURA", "ACUR", "ACRUA", "ACRURA", "ACCORD", "ACRURA"],
    "Dodge": ["DODGE", "DODGEI", "DOSGE", "DDGE"],
    "BMW": ["BMW", "BWM", "BOMW"],
    "Lincoln": ["LINCOLN", "LINOLN", "LINC", "LINCLN"],
    "Jeep": ["JEEP", "JEEK", "JEEF", "JEEPQ", "JEE"],
    "Jaguar": ["JAGUAR", "JAG", "JAGU"],
    "Porsche": ["PORSCHE", "PORSCHE"],
    "Audi": ["AUDI"],
    "Mitsubishi": [
        "MITSUBISHI",
        "MITSUBIHI",
        "MITSUBIHI",
        "MITS",
        "MITSUBIHI",
        "MITSH",
        "MITSUH",
        "MITSUBASHI",
        "MITSU",
        "MITSUBIHI",
        "MITSB",
        "MISTUBISHI",
        "MISTUBISHI",
        "MITSUBISH",
        "MIST",
        "MITSUBISHU",
        "MITSUBISH",
        "MITSUBISI",
    ],
    "Porsche": ["PORSCHE", "PORSCHE"],
    "Audi": ["AUDI", "AUD"],
    "Mercedes": [
        "MERCEDES",
        "MERZ BENZ",
        "MERCEDEZ",
        "MERCEDESBENZ",
        "MERC BENZ",
        "MERCEDS BENZ",
        "MERCADES",
        "MECEDES",
        "MERCVAL2013",
        "MERCURY",
        "MERCER",
        "MERCZ",
        "MERZ",
        "MERK",
        "MERECEDES",
        "MERCERY",
        "MERCADES",
        "MERCADES",
        "MERCADES",
        "MERCADES",
        "MERCVAL2013",
        "MERCADES",
        "MERCERY",
    ],
    "SAAB": ["SAAB", "SAA"],
    "Cadillac": ["CADILLAC", "CADI", "CADDI", "CADIALLAC", "CADDILLAC"],
    "Lobo": ["LOBO"],
    "Lamborghini": ["LAMBORGHINI", "LAMBO"],
    "Subaru": ["SUBARU", "SUBA", "SUBUARU"],
    "Buick": ["BUICK", "BRUICK", "BUIK", "BUK"],
    "Lotus": ["LOTUS", "LOTU"],
    "Rolls Royce": ["ROLLS ROYCE"],
    "Tesla": ["TESLA", "TESCA"],
    "Range Rover": ["RANGE ROVER", "RANG ROVER", "RANGEROVER"],
    "Mini": ["MINI", "MINN", "MNNI", "MINICOOP", "MINI COOPER"],
    "Land Rover": ["LNDR"],
    "Plymouth": ["PLYMOUTH", "PLYM"],
}

In [ ]:
create_yaml(car_variations, CONFIG_MAKE, "Make")

In [ ]:
data["Make_clean"] = data["Make"]

In [ ]:
replace_with_best_match(data, "Make_clean", car_variations)

print(data["Make_clean"])

In [ ]:
make_mapping_hard_coded = {
    "mercedes": ["MERZ BENZ", "BENZ", "MBENZ"],
    "isuzu": ["ISU", "ISUZ"],
    "volkswagen": ["VW", "WV"],
}

In [ ]:
create_yaml(make_mapping_hard_coded, CONFIG_HARDCODED, "Make")

In [ ]:
data = replace_with_hard(data, dict_hard=make_mapping_hard_coded, column="Make_clean")

In [ ]:
data["Make_clean"].value_counts().head(25)

In [ ]:
data = categorize_top_n(data, "Make_clean", 20)

In [ ]:
data["Make_clean"].value_counts()

### Grouping color column

In [ ]:
color_groups = {
    "DARK COLORS": ["BLACK", "GRAY", "BLUE DARK", "GREEN DK", "BROWN"],
    "LIGHT COLORS": ["SILVER", "WHITE", "BLUE LIGHT", "TAN", "BEIGE", "CREAM"],
    "BOLD COLORS": [
        "RED",
        "MAROON",
        "GREEN",
        "GOLD",
        "BLUE",
        "GREEN LGT",
        "YELLOW",
        "ORANGE",
        "BRONZE",
        "PURPLE",
        "MULTICOLOR",
        "PINK",
        "COPPER",
        "CHROME",
        "CAMOUFLAGE",
    ],
    "?": ["?"],
}

In [ ]:
create_yaml(color_groups, CONFIG_HARDCODED, "Color")

In [ ]:
data = replace_with_hard(data, dict_hard=color_groups, column="Color")

In [ ]:
data["Color"].value_counts()

### Grouping VehicleType column

In [ ]:
vehicle_type_groups = {
    "Standard Vehicles": ["02 - Automobile", "03 - Station Wagon", "04 - Limousine"],
    "Motorcycles": ["01 - Motorcycle", "19 - Moped"],
    "Trucks": [
        "05 - Light Duty Truck",
        "06 - Heavy Duty Truck",
        "20 - Commercial Rig",
    ],
    "Special Purpose": [
        "21 - Tandem Trailer",
        "25 - Utility Trailer",
        "09 - Farm Vehicle",
        "07 - Truck/Road Tractor",
        "27 - Farm Equipment",
        "10 - Transit Bus",
        "12 - School Bus",
        "11 - Cross Country Bus",
        "14 - Ambulance(Non-Emerg)",
    ],
    "other": ["28 - Other", "24 - Camper", "26 - Boat Trailer", "08 - Recreational Vehicle"],
    "?": ["29 - Unknown"],
}

In [ ]:
create_yaml(vehicle_type_groups, CONFIG_HARDCODED, "VehicleType")

In [ ]:
data = replace_with_hard(data, dict_hard=vehicle_type_groups, column="VehicleType")

In [ ]:
data["VehicleType"].value_counts()